# Imports

In [1]:
import os
import urllib.request
from pathlib import Path
import PIL
from PIL import Image
import cv2
from IPython.display import Image

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
    ###################################################################
    from keract import display_activations
    # The image path
    img_path = '../data/Yoga-82/YOGA_downloads_withoutClipart/Akarna_Dhanurasana/40.jpg'
    # Preprocessing the image for the model
    x = preprocess_image(img_path=img_path,model=model,resize=target_size)
    # Generate the activations 
    activations = get_activations(model, x)
    ###################################################################

In [ ]:
def process_img(df):
    num = df.shape[0]
    
    data = np.zeros((num,224,224,3),dtype='float32')

    for i in range(num):
        path = df['YogaPoses'].iloc[i] + '/' + df['ImageNumbers'].iloc[i]
        img_path = '../data/Yoga-82/YOGA_downloads_withoutClipart/'
        imgs = img_path + path
        
        
        image = Image.open(imgs).convert("RGB")
        image = image.resize((224,224), Image.ANTIALIAS)
        data[i][:][:][:] = image
    
    return data

In [ ]:
from keract import display_activations
from keract import get_activations


# The image path
#img_path = '../data/Yoga-82/YOGA_downloads_withoutClipart/Akarna_Dhanurasana/40.jpg'
# Preprocessing the image for the model
x = process_img(Activation_df)
# Generate the activations 
model = dense201_hirar_new()
activations = get_activations(model, x)
display_activations(activations, save=False)

In [ ]:
log_df = pd.read_csv(path+'CSVLogger_hirarnew.csv')
log_df

In [ ]:
def correctLabels_82(classification_report82_V1):
    classification_report82_V1 = class_report(path + 'classification_report82.csv').reset_index(drop=True)
    classification_report82_V1['index1'] = classification_report82_V1.index
    classification_report82_V1 = classification_report82_V1.replace({"index1": trans82})
    classification_report82_V1.iloc[-3:, classification_report82_V1.columns.get_loc('index1')] = ['accuracy', 'macro avg', 'weighted avg']
    classification_report82_V1 = classification_report82_V1.set_index('index1')
    return classification_report82_V1

#class report as pandas df
def class_report(path):
    report= pd.read_csv(path).T
    report.columns = report.iloc[0]
    report.drop(index=report.index[0], axis=0, inplace=True)
    report.columns.name = None

    return report

path = '../Results/dense201_hirar_new/'
classification_reportV3_6 = class_report(path + 'classification_report6.csv')
classification_reportV3_6   #0.219512

#classification_reportV3_20 = class_report(path + 'classification_report20.csv') 
#classification_reportV3_20 # 0.036585
#classification_report82_V3 = correctLabels_82(class_report(path + 'classification_report82.csv'))
#classification_report82_V3 #0,012195

In [ ]:
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np
import cv2

func_model = dense201_hirar_6same20()


class GradCAM:
    def __init__(self, model, classIdx, layerName=None):
        # store the model, the class index used to measure the class
        # activation map, and the layer to be used when visualizing
        # the class activation map
        self.model = model
        self.classIdx = classIdx
        self.layerName = layerName
        # if the layer name is None, attempt to automatically find
        # the target output layer
        if self.layerName is None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):
        # attempt to find the final convolutional layer in the network
        # by looping over the layers of the network in reverse order
        for layer in reversed(self.model.layers):
            # check to see if the layer has a 4D output
            if len(layer.output_shape) == 4:
                return layer.name
        # otherwise, we could not find a 4D layer so the GradCAM
        # algorithm cannot be applied
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")


    def compute_heatmap(self, image, eps=1e-8):
        # construct our gradient model by supplying (1) the inputs
        # to our pre-trained model, (2) the output of the (presumably)
        # final 4D layer in the network, and (3) the output of the
        # softmax activations from the model
        gradModel = Model(
            inputs=[self.model.inputs],
            outputs=[self.model.get_layer(self.layerName).output, self.model.output])

        # record operations for automatic differentiation
        with tf.GradientTape() as tape:
            # cast the image tensor to a float-32 data type, pass the
            # image through the gradient model, and grab the loss
            # associated with the specific class index
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            
            loss = predictions[:, tf.argmax(predictions[0])]
    
        # use automatic differentiation to compute the gradients
        grads = tape.gradient(loss, convOutputs)

        # compute the guided gradients
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads
        # the convolution and guided gradients have a batch dimension
        # (which we don't need) so let's grab the volume itself and
        # discard the batch
        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]

        # compute the average of the gradient values, and using them
        # as weights, compute the ponderation of the filters with
        # respect to the weights
        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

        # grab the spatial dimensions of the input image and resize
        # the output class activation map to match the input image
        # dimensions
        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))
        # normalize the heatmap such that all values lie in the range
        # [0, 1], scale the resulting values to the range [0, 255],
        # and then convert to an unsigned 8-bit integer
        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")
        # return the resulting heatmap to the calling function
        return heatmap

    def overlay_heatmap(self, heatmap, image, alpha=0.5,
                        colormap=cv2.COLORMAP_VIRIDIS):
        # apply the supplied color map to the heatmap and then
        # overlay the heatmap on the input image
        heatmap = cv2.applyColorMap(heatmap, colormap)
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)
        # return a 2-tuple of the color mapped heatmap and the output,
        # overlaid image
        return (heatmap, output)

    #Prediction

image = cv2.imread('../data/Yoga-82/YOGA_downloads_withoutClipart/Akarna_Dhanurasana/2.jpg')
image = cv2.resize(image, (32, 32))
image = image.astype('float32') / 255
image = np.expand_dims(image, axis=0)

preds = func_model.predict(image) 
i = np.argmax(preds[0])

#To get the layer's name of the model

for idx in range(len(func_model.layers)):
    print(func_model.get_layer(index = idx).name)



In [ ]:
# we picked `block5c_project_con` layer 
#Passing to GradCAM class

icam = GradCAM(func_model, i, 'block5c_project_conv') 
heatmap = icam.compute_heatmap(image)
heatmap = cv2.resize(heatmap, (32, 32))

image = cv2.imread('/content/dog.jpg')
image = cv2.resize(image, (32, 32))
print(heatmap.shape, image.shape)

(heatmap, output) = icam.overlay_heatmap(heatmap, image, alpha=0.5)

#Visualization

fig, ax = plt.subplots(1, 3)

ax[0].imshow(heatmap)
ax[1].imshow(image)
ax[2].imshow(output)